In [12]:
import json
from utils.graph_creation import create_hetero_graph

# Dataset Statistics and Analysis

In [13]:
with open("./data/all_users.json", "r") as f:
    all_users = json.load(f)

print(f"Total users: {len(all_users)}")

num_ascends = 0
for user in all_users.values():
    num_ascends += len(user.get("problems", []))
print(f"Total ascends: {num_ascends}")

num_ascends = 0
for user in all_users.values():
    num_ascends += user.get("problems_sent", 0)
print(f"Total ascends: {num_ascends}")

Total users: 25865
Total ascends: 1668865
Total ascends: 1807394


In [14]:
with open("./data/all_problems.json", "r") as f:
    all_problems = json.load(f)

print(f"Total problems: {len(all_problems)}")

Total problems: 38663


# Graph Statistics and Analysis

In [5]:
hetero_graph = create_hetero_graph()

In [6]:
graph = create_hetero_graph(holds_as_nodes=False)

In [9]:
import networkx as nx
import numpy as np
from collections import Counter
import torch
import random

def analyze_hetero_graph(data, name="Graph"):
    """
    Analyze a HeteroData graph and compute various statistics.
    """
    print(f"\n{'='*60}")
    print(f"Analysis for: {name}")
    print(f"{'='*60}")
    
    # 1. Number of nodes of each type
    print("\n--- Number of Nodes per Type ---")
    total_nodes = 0
    node_types = data.node_types
    for node_type in node_types:
        num_nodes = data[node_type].x.shape[0]
        print(f"  {node_type}: {num_nodes:,}")
        total_nodes += num_nodes
    print(f"  Total: {total_nodes:,}")
    
    # 2. Number of edges of each type
    print("\n--- Number of Edges per Type ---")
    total_edges = 0
    edge_types = data.edge_types
    for edge_type in edge_types:
        num_edges = data[edge_type].edge_index.shape[1]
        print(f"  {edge_type}: {num_edges:,}")
        total_edges += num_edges
    # Divide by 2 since we have reverse edges
    print(f"  Total (including reverse): {total_edges:,}")
    print(f"  Total (unique): {total_edges // 2:,}")
    
    # Convert to homogeneous NetworkX graph for global metrics
    print("\n--- Converting to NetworkX (this may take a moment) ---")
    
    G = nx.Graph()
    
    # Create node mappings to global indices
    node_offset = {}
    current_offset = 0
    for node_type in node_types:
        node_offset[node_type] = current_offset
        num_nodes = data[node_type].x.shape[0]
        # Add nodes with type attribute
        G.add_nodes_from(
            [(current_offset + i, {'type': node_type}) for i in range(num_nodes)]
        )
        current_offset += num_nodes
    
    # Add edges (only non-reverse to avoid duplicates)
    for edge_type in edge_types:
        src_type, rel, dst_type = edge_type
        if rel.startswith('rev_'):
            continue  # Skip reverse edges
        
        edge_index = data[edge_type].edge_index.numpy()
        src_offset = node_offset[src_type]
        dst_offset = node_offset[dst_type]
        
        edges = [
            (src_offset + edge_index[0, i], dst_offset + edge_index[1, i])
            for i in range(edge_index.shape[1])
        ]
        G.add_edges_from(edges)
    
    print(f"  NetworkX graph: {G.number_of_nodes():,} nodes, {G.number_of_edges():,} edges")
    
    # 3. Number of connected components
    print("\n--- Connected Components ---")
    num_components = nx.number_connected_components(G)
    print(f"  Number of connected components: {num_components:,}")
    
    # Get largest component for diameter calculation
    if num_components > 1:
        largest_cc = max(nx.connected_components(G), key=len)
        G_largest = G.subgraph(largest_cc).copy()
        print(f"  Largest component size: {G_largest.number_of_nodes():,} nodes")
    else:
        G_largest = G
    
    # 4. Graph Diameter (on largest connected component)
    # For large graphs, exact diameter is expensive - use approximation
    print("\n--- Graph Diameter (largest component) ---")
    if G_largest.number_of_nodes() > 10000:
        # Use approximation for large graphs
        try:
            diameter_approx = nx.approximation.diameter(G_largest)
            print(f"  Approximate diameter: {diameter_approx}")
        except Exception as e:
            print(f"  Diameter computation failed: {e}")
    else:
        diameter = nx.diameter(G_largest)
        print(f"  Exact diameter: {diameter}")
    
    # 5. Graph Density
    print("\n--- Graph Density ---")
    density = nx.density(G)
    print(f"  Density: {density:.6e}")
    
    # 6. Average Clustering Coefficient
    # For large graphs, sample nodes to compute approximate clustering
    print("\n--- Average Clustering Coefficient ---")
    if G.number_of_nodes() > 50000:
        # Sample nodes for approximation
        sample_size = min(10000, G.number_of_nodes())
        sampled_nodes = random.sample(list(G.nodes()), sample_size)
        avg_clustering = nx.average_clustering(G, nodes=sampled_nodes)
        print(f"  Approximate average clustering (sample of {sample_size:,}): {avg_clustering:.6f}")
    else:
        avg_clustering = nx.average_clustering(G)
        print(f"  Average clustering coefficient: {avg_clustering:.6f}")
    
    # 7. Node Degree Distribution for each node type
    print("\n--- Node Degree Distribution per Type ---")
    for node_type in node_types:
        offset = node_offset[node_type]
        num_nodes = data[node_type].x.shape[0]
        node_ids = range(offset, offset + num_nodes)
        
        degrees = [G.degree(n) for n in node_ids]
        degrees = np.array(degrees)
        
        print(f"\n  {node_type}:")
        print(f"    Min degree: {degrees.min()}")
        print(f"    Max degree: {degrees.max()}")
        print(f"    Mean degree: {degrees.mean():.2f}")
        print(f"    Median degree: {np.median(degrees):.2f}")
        print(f"    Std degree: {degrees.std():.2f}")
        
        # Show degree distribution (top 5 most common)
        degree_counts = Counter(degrees)
        most_common = degree_counts.most_common(5)
        print(f"    Top 5 most common degrees: {most_common}")
    
    return G

In [10]:
# Analyze the graph WITH hold nodes
G_with_holds = analyze_hetero_graph(hetero_graph, "HeteroGraph WITH hold nodes")


Analysis for: HeteroGraph WITH hold nodes

--- Number of Nodes per Type ---
  user: 25,826
  problem: 34,572
  hold: 198
  Total: 60,596

--- Number of Edges per Type ---
  ('user', 'rates', 'problem'): 1,627,580
  ('problem', 'rev_rates', 'user'): 1,627,580
  ('problem', 'contains', 'hold'): 304,852
  ('hold', 'rev_contains', 'problem'): 304,852
  Total (including reverse): 3,864,864
  Total (unique): 1,932,432

--- Converting to NetworkX (this may take a moment) ---
  NetworkX graph: 60,596 nodes, 1,932,432 edges

--- Connected Components ---
  Number of connected components: 1

--- Graph Diameter (largest component) ---
  Approximate diameter: 6

--- Graph Density ---
  Density: 1.052576e-03

--- Average Clustering Coefficient ---
  Approximate average clustering (sample of 10,000): 0.000000

--- Node Degree Distribution per Type ---

  user:
    Min degree: 1
    Max degree: 3676
    Mean degree: 63.02
    Median degree: 20.00
    Std degree: 118.73
    Top 5 most common degrees: 

In [11]:
# Analyze the graph WITHOUT hold nodes
G_without_holds = analyze_hetero_graph(graph, "HeteroGraph WITHOUT hold nodes")


Analysis for: HeteroGraph WITHOUT hold nodes

--- Number of Nodes per Type ---
  user: 25,826
  problem: 34,572
  Total: 60,398

--- Number of Edges per Type ---
  ('user', 'rates', 'problem'): 1,627,580
  ('problem', 'rev_rates', 'user'): 1,627,580
  Total (including reverse): 3,255,160
  Total (unique): 1,627,580

--- Converting to NetworkX (this may take a moment) ---
  NetworkX graph: 60,398 nodes, 1,627,580 edges

--- Connected Components ---
  Number of connected components: 6
  Largest component size: 60,390 nodes

--- Graph Diameter (largest component) ---
  Approximate diameter: 7

--- Graph Density ---
  Density: 8.923483e-04

--- Average Clustering Coefficient ---
  Approximate average clustering (sample of 10,000): 0.000000

--- Node Degree Distribution per Type ---

  user:
    Min degree: 1
    Max degree: 3676
    Mean degree: 63.02
    Median degree: 20.00
    Std degree: 118.73
    Top 5 most common degrees: [(1, 1893), (2, 1529), (3, 1290), (4, 1011), (5, 868)]

  pr